In [1]:
# from google.colab import drive
# drive.mount('/content/drive')
# %cd drive/MyDrive/aml4

In [2]:
import os
import glob
import time
import numpy as np
import pandas as pd
import tensorflow as tf

In [3]:
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard, ReduceLROnPlateau

In [4]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import fbeta_score, make_scorer

# scoring
from sklearn.metrics import accuracy_score,precision_score,recall_score,confusion_matrix,roc_curve,roc_auc_score,auc, classification_report

In [5]:
import matplotlib.pyplot as plt
from matplotlib import style
from collections import defaultdict
# configure
# sets matplotlib to inline and displays graphs below the corressponding cell.
%matplotlib inline
style.use('fivethirtyeight')

In [6]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [7]:
from snapshots.snapshot_ens import *
from baseline.baseline import Baseline
from improve_snapshot.swa.tfkeras import SWA

# HELPERS

In [8]:
def model_fit_with_grid_search_nested_cv(model,parameters,fit_params, X, y, df_name, round, folds = 10, score = 'accuracy', verbose = 0 ):

    start = time.time()

    cv_inner = StratifiedKFold(n_splits=int(folds/3), shuffle=True, random_state=1)
    cv_outer = StratifiedKFold(n_splits=folds, shuffle=True, random_state=1)

    # create the sklearn model for the network
    from baseline.baseline import Baseline
    bl = Baseline(X.shape[1], len(np.unique(y)))
    model_init_batch_epoch_CV = KerasClassifier(build_fn=bl.create_model, verbose=1)

    clf = GridSearchCV(
        estimator = model_init_batch_epoch_CV,
        param_grid = parameters,
        cv = cv_inner,
        scoring = score, 
        verbose = verbose,
        n_jobs = 1,
        refit=True
    )

    # Non-nested parameter search and scoring
    grid_result = clf.fit(X, y)
    outer_scores[round] = grid_result.best_score_
    nested_score = cross_val_score(clf, X, y, scoring=score, cv=cv_outer, n_jobs=-1, fit_params=fit_params)
    nested_scores[round] = nested_score.mean()

    if verbose > 0:
        print("--- Ellapsed time: %s seconds ---" % (time.time() - start))
        print('Best params: ',grid_result.best_params_)
        print('Best score (%s)' % nested_score, grid_result.best_score_)
        print('%s Nested Cross Validation Accuracy: %.3f (%.3f)' % (df_name, np.mean(nested_score),                                                                             np.std (nested_score)))
    return clf.best_estimator_, grid_result.best_params_, grid_result.best_score_, outer_scores, nested_scores, time.time() - start

In [9]:
def plot_nested_cv(model_name, nested_scores, outer_scores):
    # Plot scores on each trial for nested and non-nested cross-validation
    plt.style.use('seaborn')
    plt.tight_layout()
    plt.figure(figsize=(10,5))
    outer_scores_line, = plt.plot(outer_scores, color='orange')
    nested_line, = plt.plot(nested_scores, color='steelblue')
    plt.ylabel("Score", fontsize="14")
    plt.legend([outer_scores_line, nested_line],
            ["Non-Nested CV", "Nested CV"],
            bbox_to_anchor=(0, .4, .5, 0))
    plt.title("Non-Nested vs Nested Cross-Validation using "+model_name,
            x=.5, y=1.1, fontsize="15")
    # Save the plot
    plt.savefig("nested-vs-non-nested.png", dpi=150)
    #Take the difference from the non-nested and nested scores
    score_difference = outer_scores - nested_scores

    print("Avg. difference of {:6f} with std. dev. of {:6f}."
        .format(score_difference.mean(), score_difference.std()))

# TRAINING

# EVALUATION HELPERS

In [10]:
def predict(model,X,y):
    df_result = pd.DataFrame(columns = ['TrueClass','Predicted'])
    df_result.Predicted = model.predict(X)
    df_result.TrueClass = y.values.ravel()
    return df_result

In [11]:
def plot_confusion_matrix(df,title,labels = ['Negative', 'Positive'],dataset_type = 'Test'):
    conf_matrix = confusion_matrix(df.TrueClass, df.Predicted)
    plt.figure(figsize=(8, 8))
    sns.heatmap(conf_matrix, xticklabels=labels, yticklabels=labels, annot=True, fmt="d");
    plt.title('{0} - Confusion matrix - {1} set'.format(title,dataset_type), fontsize = 20)
    plt.xlabel('Predicted class')
    plt.ylabel('True class')
    plt.show()
    return conf_matrix.ravel()

In [12]:
def evalute_model_performance(model, model_name, X, y, df_name='none'):
    df_result = predict(model,X,y)
    class_report = classification_report(df_result.TrueClass, df_result.Predicted, output_dict= True)
    accuracy = class_report['accuracy']
    recall = class_report['macro avg']['recall']
    precision = class_report['macro avg']['precision']
    auc_pr = round(auc(recall, precision),2)
    f1 = class_report['macro avg']['f1-score']
    f2 = fbeta_score(df_result.TrueClass, df_result.Predicted,beta = 2, average='macro')

    naive_probs = [0 for _ in range(len(y))]
    
    probs = model.predict_proba(X)
    probs = probs[:, 1]

    naive_auc = roc_auc_score(y, naive_probs)
    model_auc = roc_auc_score(y, probs)

    print('No Skill: ROC AUC=%.3f' % (naive_auc))
    print(model_name,': ROC AUC=%.3f' % (model_auc))

    naive_fpr, naive_tpr, _ = roc_curve(y, naive_probs)
    model_fpr, model_tpr, _ = roc_curve(y, probs)

    print('')
    print('Performance Report: ')

    print('Accuracy: %1.3f' % accuracy) #a
    print('TPR: %1.3f' % model_tpr) #b
    print('FPR: %1.3f' % model_fpr) #c
    print('Precision: %1.3f' % precision) #d
    print('AUC: %1.3f' % model_auc) #e
    print('AUC-PR: %1.3f' % auc_pr) #f
    print('Recall: %1.3f' % recall)
    print('F1: %1.3f' % f1)
    print('F2: %1.3f' % f2)

    print('')

    plot_confusion_matrix(df_result, model_name)

    try:
        plot_ROC(model, model_name, model_fpr, model_tpr, naive_fpr, naive_tpr)
    except:
        print('Could not print ROC AUC curve.')
    new_row = {"Dataset Name":df_name, "Algorithm Name":model_name, "Cross Validation":1, "Hyper-Parameters Values":0.5, "Accuracy":accuracy, "TPR":model_tpr, "FPR":model_fpr, "Precision":precision, "AUC":auc, "AUC-PR":auc_pr, "Train Time":5.32, "Inference Time":0.2}
    EVAL_DF = EVAL_DF.append(new_row, ignore_index=True)

In [13]:
def plot_ROC(model,model_name, model_fpr, model_tpr, naive_fpr, naive_tpr):
    plt.plot(naive_fpr, naive_tpr, linestyle='--', label='Naive')
    plt.plot(model_fpr, model_tpr, marker='.', label=model_name)
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.legend()
    plt.show()

In [14]:
def run_training_main_loop(X, y, df_name, rounds):
    global models_to_run, models_param_grid, fit_params, outer_scores, nested_scores
    init_models_and_params(X, y)
    print('='*20,df_name,18*'=','Shapes:',X.shape[1], len(np.unique(y)))
    for i,model in enumerate(models_to_run):
        model_name = model.__class__.__name__+' '+str(i)   
        print('-'*10,model_name+str(i),10*'-')
        
        #Clear scores arrays
        outer_scores = np.zeros(rounds)
        nested_scores = np.zeros(rounds)
        
        for _round in range(rounds):
            best_model,best_model_params,best_model_score, outer_scores, nested_scores, et = model_fit_with_grid_search_nested_cv( model,
                                                    models_param_grid,
                                                    fit_params[i],
                                                    X,
                                                    y,
                                                    df_name,
                                                    folds = 10,
                                                    score = 'accuracy',
                                                    round = _round, 
                                                    verbose = 0
                                                )

            best_models_dict[i].append((best_model,best_model_params, best_model_score, outer_scores, nested_scores, et))
            # inner loop
        plot_nested_cv(model_name, nested_scores, outer_scores)
        print('-'*30)
    print(80*'=')
    print(3*'\n')    

# CALLBACKS

In [15]:
log_dir = "logs/fit/" + pd.datetime.now().strftime("%H%M%S")

tensor_board = TensorBoard(
    log_dir=log_dir, histogram_freq=0, write_graph=True,
    write_images=True, update_freq='epoch', profile_batch=2,
    embeddings_freq=0, embeddings_metadata=None
)

# define snapshot callback
snp = Snapshot('snapshots', nb_epochs=6, verbose=1, nb_cycles=2)

epochs = 50
start_epoch = epochs//2

# define swa callback
swa = SWA(start_epoch=start_epoch, 
          lr_schedule='constant', 
          swa_lr=0.01, 
          verbose=1)

callbacks = [
            #  ReduceLROnPlateau(monitor='val_loss', patience=5, cooldown=0),
             EarlyStopping(monitor='val_acc', min_delta=1e-6, patience=15),
             tensor_board,
             snp,
             swa
             ]

<ipython-input-15-e1e10c88da56>:1: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  log_dir = "logs/fit/" + pd.datetime.now().strftime("%H%M%S")


# MAIN

In [16]:
#Number of rounds
rounds = 50

#Create arrays to store the scores
outer_scores = np.zeros(rounds)
nested_scores = np.zeros(rounds)

In [17]:
column_names = ["Dataset Name", "Algorithm Name", "Cross Validation","Hyper-Parameters Values", "Accuracy", "TPR", "FPR", "Precision", "AUC", "AUC-PR", "Training", "Train Time", "Inference Time"]
EVAL_DF = pd.DataFrame(columns = column_names)

# DATASETS

In [18]:
datasets_dict = defaultdict(list)
ds = os.listdir('datasets')[2:10]

for df_name in ds:
    # make dataset
    df = pd.read_csv('datasets/%s' % df_name)
    X, y = df.iloc[:, :-1], df.iloc[:, -1]
    # y = to_categorical(y)
    datasets_dict[df_name[:-4]] = (X, y)

# MODELING

In [19]:
models_to_run = []
models_param_grid = dict()
fit_params = {}
best_models_dict = defaultdict(list)
optimizer = SGD(lr=0.1, momentum=0.9, nesterov=True)

/Users/avihais/Library/Python/3.9/lib/python/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


In [20]:
def init_models_and_params(X, y): 
        global models_to_run, models_param_grid, fit_params   
        models_to_run = [
                Baseline(x_in=X.shape[1], x_out=len(np.unique(y))).baseline(),
                Baseline(x_in=X.shape[1], x_out=len(np.unique(y))).baseline(),
                Baseline(x_in=X.shape[1], x_out=len(np.unique(y))).baseline(),
                ]
        
        # we choose the initializers that came at the top in our previous               cross-validation!!
        init_mode = ['glorot_uniform', 'uniform', 'lecun_uniform'] 
        batches = [32, 64, 128]
        epochs = [10, 20, 30]

        models_param_grid = dict(epochs=epochs, batch_size=batches, init=init_mode)

        fit_params = [
                {'callbacks': callbacks[:-2]}, #baseline
                {'callbacks': callbacks[:-1]}, #snapshots-ensemble
                {'callbacks': callbacks[:-2]+[callbacks[-1]]} #stochastic-weight-averaging
        ]

        return len(models_to_run) == 3

In [22]:
del model
print('Loading ensemble...')
keep_last = 2
model = load_ensemble('snapshots')
model.compile(
    optimizer=SGD(lr=0.1, momentum=0.9, nesterov=True),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
metrics = model.evaluate(x_test, y_test)
print(metrics)

NameError: name 'model' is not defined

In [26]:
%%capture cap --no-stderr
print("FILE WRITING")
# main loop
with open('output.txt', 'w') as f:
    f.write(cap.stdout)
    [run_training_main_loop(X, y, df_name, rounds) for df_name, (X, y) in datasets_dict.items()]

 will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '
/Users/avihais/Library/Python/3.9/lib/python/site-packages/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '
/Users/avihais/Library/Python/3.9/lib/pytho

KeyboardInterrupt: 